In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
X=pd.read_csv("../Dataset/nepal_earthquake_train_values.csv")
Y=pd.read_csv("../Dataset/nepal_earthquake_train_labels.csv")
Test=pd.read_csv("../Dataset/nepal_earthquake_test_values.csv")

In [16]:
Test.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,...,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,...,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,...,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,...,0,0,0,0,0,0,0,0,0,0


In [17]:
X.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,...,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,...,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,...,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,...,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,...,0,0,0,0,0,0,0,0,0,0


In [18]:
X.drop(columns=["building_id"],inplace=True)
Test.drop(columns=["building_id"],inplace=True)

In [19]:
Y.drop(columns=["building_id"],inplace=True)

In [104]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder,StandardScaler,MinMaxScaler,LabelEncoder,OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split


X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=.2,random_state=42)

numaric_columns=X.select_dtypes(include=["int","float32"]).columns.tolist()
object_columns=X.select_dtypes(include=["object","category"]).columns.tolist()

numaric_pipeline=Pipeline(
    [
        ("impute",SimpleImputer(strategy="mean")),
        ("standard",StandardScaler()),
    ]
)

object_pipeline=Pipeline(
    [
        ("impute",SimpleImputer(strategy="most_frequent")),
        ('encoder',OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1))
    ],
   
)

column_transform=ColumnTransformer(
    [
        ("num",numaric_pipeline,numaric_columns),
        ("object",object_pipeline,object_columns),
    ]
)

In [114]:
import lightgbm as lgb

lgb_model = lgb.LGBMClassifier(
    objective="multiclass",
    num_class=len(np.unique(Y)),
    boosting_type='gbdt',
    learning_rate=0.05,
    n_estimators=4000,
    max_depth=25,
    num_leaves=50,
    min_data_in_leaf=30,
    min_child_samples=20,
    min_split_gain=0.01,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.5,
    reg_lambda=0.1,
    random_state=42,
    deterministic=True,
    n_jobs=-1,
    verbose=50,
    force_row_wise=True
)



model=Pipeline(
    [
        ("transform",column_transform),
        ("lgbm",lgb_model)
    ]
)


model.fit(X_train,Y_train)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.714994
[LightGBM] [Info] Total Bins 797
[LightGBM] [Info] Number of data points in the train set: 208480, number of used features: 36
[LightGBM] [Info] Start training from score -2.346413
[LightGBM] [Info] Start training from score -0.562637
[LightGBM] [Info] Start training from score -1.094868
[LightGBM] [Debug] Trained a tree with leaves = 50 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 50 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 50 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 50 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 50 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 50 and depth

Pipeline(steps=[('transform',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('standard',
                                                                   StandardScaler())]),
                                                  ['geo_level_1_id',
                                                   'geo_level_2_id',
                                                   'geo_level_3_id',
                                                   'count_floors_pre_eq', 'age',
                                                   'area_percentage',
                                                   'height_percentage',
                                                   'has_superstructure_adobe_mud',
                                                   'has_superstructure_mud_mortar_stone',
                                                   'has_superstructure_...
                                                   'plan_configuration',
                                                   'legal_ownership_status'])])),
                ('lgbm',
                 LGBMClassifier(colsample_bytree=0.8, deterministic=True,
                                force_row_wise=True, learning_rate=0.05,
                                max_depth=25, min_data_in_leaf=30,
                                min_split_gain=0.01, n_estimators=4000,
                                n_jobs=-1, num_class=3, num_leaves=50,
                                objective='multiclass', random_state=42,
                                reg_alpha=0.5, reg_lambda=0.1, subsample=0.8,
                                verbose=50))])

In [115]:
model.score(X_test,Y_test)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


0.7453233821300436

In [116]:
y_pred=model.predict(Test)

c:\Users\Lenovo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30


In [117]:
x_test3=pd.read_csv("../Dataset/nepal_earthquake_test_values.csv")
df=pd.DataFrame({"building_id":x_test3['building_id'], "damage_grade":y_pred} )
df["damage_grade"]=df["damage_grade"].astype(int)
df.to_csv("modified_earthquake_damage.csv",index=False)
df.head()

,building_id,damage_grade
0,300051,3
1,99355,2
2,890251,2
3,745817,1
4,421793,3
